In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df1 = pd.read_csv ("total.csv")
# df1

In [3]:
# ---------------- 1. 필요한 열만 선택 ----------------
df = df1[["날짜", "호선", "역명", "상하구분", "기온", "체감온도", "시간강수량", "혼잡도"]].copy()

# ---------------- 2. 날짜 관련 파생 컬럼 생성 ----------------
# 날짜 컬럼: yyyymmddhh 형식 → 문자열 길이 체크
df = df[df['날짜'].astype(str).str.len() == 10]

df['년'] = df['날짜'].astype(str).str[0:4].astype(int)
df['월'] = df['날짜'].astype(str).str[4:6].astype(int)
df['일'] = df['날짜'].astype(str).str[6:8].astype(int)
df['시'] = df['날짜'].astype(str).str[8:10].astype(int)

# 2022년 1월 1일 00시 이후 데이터만 필터링
df = df[df['날짜'] >= 2022010100]
print(f"📅 2022년 이후 데이터 개수: {len(df)}")

# 계절 함수 정의
def 계절_정의(월):
    if 월 in [3, 4, 5]: return '봄'
    elif 월 in [6, 7, 8]: return '여름'
    elif 월 in [9, 10, 11]: return '가을'
    else: return '겨울'

df['계절'] = df['월'].apply(계절_정의)
df['시간대'] = df['시'].apply(lambda x: f"{x:02d}시")

# 시간대 컬럼 만들기 (예: 07시, 08시)
df['시간대'] = df['시'].apply(lambda x: f"{x:02d}시")

📅 2022년 이후 데이터 개수: 11607871


In [4]:
# ---------------- 3. 강수 상태 파생 ----------------
def 비상태_정의(강수량):
    if 강수량 < 1: return '강수량 없음'
    elif 1 <= 강수량 < 15: return '약한 비 or 눈'
    elif 강수량 >= 15: return '강한 비 or 눈'

df['시간강수량_상태'] = df['시간강수량'].apply(비상태_정의)

df

,날짜,호선,역명,상하구분,기온,체감온도,시간강수량,혼잡도,년,월,일,시,계절,시간대,시간강수량_상태
5804733,2022010100,1,서울역,상선,-9.4,-12.8,0.0,0,2022,1,1,0,겨울,00시,강수량 없음
5804734,2022010101,1,서울역,상선,-9.4,-10.4,0.0,0,2022,1,1,1,겨울,01시,강수량 없음
5804735,2022010105,1,서울역,상선,-9.7,-10.8,0.0,2,2022,1,1,5,겨울,05시,강수량 없음
5804736,2022010106,1,서울역,상선,-9.4,-11.0,0.0,5,2022,1,1,6,겨울,06시,강수량 없음
5804737,2022010107,1,서울역,상선,-10.1,-10.9,0.0,4,2022,1,1,7,겨울,07시,강수량 없음
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17412599,2023123119,8,남위례,하선,0.6,0.0,0.0,18,2023,12,31,19,겨울,19시,강수량 없음
17412600,2023123120,8,남위례,하선,0.0,-0.6,0.0,17,2023,12,31,20,겨울,20시,강수량 없음
17412601,2023123121,8,남위례,하선,-0.6,-1.1,0.0,21,2023,12,31,21,겨울,21시,강수량 없음
17412602,2023123122,8,남위례,하선,-0.8,-1.3,0.0,18,2023,12,31,22,겨울,22시,강수량 없음


In [5]:
# ---------------- 4. 이상치 및 결측치 처리 ----------------

# (1) 시간강수량 -99 → NaN (이상치 개수)
count_minus_99 = (df['시간강수량'] == -99).sum()
print(f"❗ '-99' 값 개수 (시간강수량): {count_minus_99}")
df['시간강수량'] = df['시간강수량'].replace(-99, np.nan)

# (1-2) 시간강수량 결측치 개수 (NaN 포함)
missing_rain = df['시간강수량'].isna().sum()
print(f"❗ 시간강수량 결측치 개수 (NaN): {missing_rain}")

# (1-3) 시간강수량 결측치 제거
before_rain_nan_drop = len(df)
df = df.dropna(subset=['시간강수량'])
removed_rain_nan = before_rain_nan_drop - len(df)
print(f"🧹 시간강수량 결측치 제거 수: {removed_rain_nan}")

# (2) 기온, 체감온도 이상치 제거 (-30 < 값 < 50)
before_temp = len(df)
df = df[(df['기온'] > -30) & (df['기온'] < 50)]
df = df[(df['체감온도'] > -30) & (df['체감온도'] < 50)]
removed_temp = before_temp - len(df)
print(f"🌡️ 기온/체감온도 이상치 제거 수: {removed_temp}")

# (3) 시간강수량 이상치 제거 (0 이상만 허용)
before_rain = len(df)
df = df[df['시간강수량'] >= 0]
removed_rain_outlier = before_rain - len(df)
print(f"🌧️ 시간강수량 이상치(0 미만) 제거 수: {removed_rain_outlier}")

# (4) 혼잡도 결측치 제거
missing_cong = df['혼잡도'].isna().sum()
print(f"❗ 혼잡도 결측치 개수: {missing_cong}")
before_cong = len(df)
df = df.dropna(subset=['혼잡도'])
removed_cong_nan = before_cong - len(df)
print(f"🧹 혼잡도 결측치 제거 수: {removed_cong_nan}")

# (5) 중복 제거
before_dup = len(df)
df = df.drop_duplicates()
removed_dup = before_dup - len(df)
print(f"🔁 중복 제거 수: {removed_dup}")

# 최종 데이터 크기
print(f"\n✅ 최종 데이터 개수: {len(df)}")

❗ '-99' 값 개수 (시간강수량): 0
❗ 시간강수량 결측치 개수 (NaN): 51106
🧹 시간강수량 결측치 제거 수: 51106
🌡️ 기온/체감온도 이상치 제거 수: 464
🌧️ 시간강수량 이상치(0 미만) 제거 수: 0
❗ 혼잡도 결측치 개수: 0
🧹 혼잡도 결측치 제거 수: 0
🔁 중복 제거 수: 759232

✅ 최종 데이터 개수: 10797069


In [6]:
# ---------------- 5. 통계 요약 ----------------
print(f"\n📉 최종적으로 줄어든 데이터 개수: {len(df1) - len(df)}")
print(f"🎯 이상치 제거 후 남은 데이터 개수: {len(df)}")

print("\n📊 '기온' 범위 확인:")
print(f"최소값: {df['기온'].min()} / 최대값: {df['기온'].max()}")

print("\n📊 '체감온도' 범위 확인:")
print(f"최소값: {df['체감온도'].min()} / 최대값: {df['체감온도'].max()}")

print("\n📊 '시간강수량' 범위 확인:")
print(f"최소값: {df['시간강수량'].min()} / 최대값: {df['시간강수량'].max()}")

print("\n📊 '혼잡도' 범위 확인:")
print(f"최소값: {df['혼잡도'].min()} / 최대값: {df['혼잡도'].max()}")


📉 최종적으로 줄어든 데이터 개수: 6615535
🎯 이상치 제거 후 남은 데이터 개수: 10797069

📊 '기온' 범위 확인:
최소값: -21.0 / 최대값: 38.2

📊 '체감온도' 범위 확인:
최소값: -25.2 / 최대값: 37.0

📊 '시간강수량' 범위 확인:
최소값: 0.0 / 최대값: 136.5

📊 '혼잡도' 범위 확인:
최소값: 0 / 최대값: 289


In [7]:
# ---------------- 6. 눈 오는 겨울날 컬럼 생성 + 상관계수 분석 ----------------

# 조건: 겨울 & 시간강수량 > 0 & (기온 ≤ 0 or 체감온도 ≤ 0)
df['눈_오는_겨울날'] = (
    (df['계절'] == '겨울') &
    (df['시간강수량'] > 0) &
    ((df['기온'] <= 0) | (df['체감온도'] <= 0))
).astype(int)

# 확인용 출력
print("\n❄️ 눈 오는 겨울날 개수 (기온/체감온도 조건 포함):", df['눈_오는_겨울날'].sum())

# 겨울철 데이터만 필터링
winter_df = df[df['계절'] == '겨울']


❄️ 눈 오는 겨울날 개수 (기온/체감온도 조건 포함): 13938


In [8]:
# ---------------- 7. 계절별 강수량과 혼잡도의 상관계수 구하기 ----------------

seasons = df['계절'].unique()

for season in seasons:
    df_season = df[df['계절'] == season]

    # 비/눈 오는 날 (강수량 > 0)
    df_rain = df_season[df_season['시간강수량'] > 0]
    # 비/눈 안 오는 날 (강수량 == 0)
    df_no_rain = df_season[df_season['시간강수량'] == 0]

    # 상관계수 (강수량 vs 혼잡도)
    corr_rain = df_rain['시간강수량'].corr(df_rain['혼잡도'])

    print(f"📌 {season}")
    print(f"  - 🌧️ 비/눈 오는 날 상관계수: {corr_rain:.4f} (관측 수: {len(df_rain)})")
    print(f"  - 🌧️ 비/눈 오는 날 평균 혼잡도: {df_rain['혼잡도'].mean():.2f}")
    print(f"  - ☀️ 비/눈 안 오는 날 평균 혼잡도: {df_no_rain['혼잡도'].mean():.2f}")
    print("----------")

📌 겨울
  - 🌧️ 비/눈 오는 날 상관계수: 0.1088 (관측 수: 67538)
  - 🌧️ 비/눈 오는 날 평균 혼잡도: 24.38
  - ☀️ 비/눈 안 오는 날 평균 혼잡도: 21.87
----------
📌 봄
  - 🌧️ 비/눈 오는 날 상관계수: 0.0116 (관측 수: 112722)
  - 🌧️ 비/눈 오는 날 평균 혼잡도: 18.53
  - ☀️ 비/눈 안 오는 날 평균 혼잡도: 23.86
----------
📌 여름
  - 🌧️ 비/눈 오는 날 상관계수: 0.0071 (관측 수: 317308)
  - 🌧️ 비/눈 오는 날 평균 혼잡도: 21.97
  - ☀️ 비/눈 안 오는 날 평균 혼잡도: 22.62
----------
📌 가을
  - 🌧️ 비/눈 오는 날 상관계수: -0.0335 (관측 수: 149609)
  - 🌧️ 비/눈 오는 날 평균 혼잡도: 22.93
  - ☀️ 비/눈 안 오는 날 평균 혼잡도: 24.97
----------


In [9]:
# 결과 CSV로 저장
df.to_csv("계절별 강수량과 혼잡도 상관관계 분석.csv", encoding='utf-8-sig', index=False)

In [10]:
a = df[(df['계절'] == '겨울') & (df['시간강수량_상태'] == '강한 비 or 눈')]
a

,날짜,호선,역명,상하구분,기온,체감온도,시간강수량,혼잡도,년,월,일,시,계절,시간대,시간강수량_상태,눈_오는_겨울날
11808506,2023011501,4,대야미,상선,8.4,2.5,31.0,0,2023,1,15,1,겨울,01시,강한 비 or 눈,0
11809157,2023011501,4,대야미,하선,8.4,2.5,31.0,0,2023,1,15,1,겨울,01시,강한 비 or 눈,0
11809808,2023011501,4,반월,상선,8.4,2.5,31.0,0,2023,1,15,1,겨울,01시,강한 비 or 눈,0
11810459,2023011501,4,반월,하선,8.4,2.5,31.0,0,2023,1,15,1,겨울,01시,강한 비 or 눈,0
